<a href="https://colab.research.google.com/github/dhamu2908/DeepLearningAssignment2/blob/main/DeepLearningAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Deep Learning Assignment 2
CS24M027 Dhamodharan Muthu Muniyandi
IIT MADRAS
"""